In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [2]:
from pivottablejs import pivot_ui

In [3]:
from jupyter_core.paths import jupyter_data_dir

## Import wv1 raw data and drop columns

In [4]:
# read in raw data (3009 rows, 239 columns)
raw = pd.read_csv("project_3_raw_data_wv1.csv")

In [5]:
# inspect all the columns
# for cl in raw.columns:
#     print(cl)

In [6]:
# caseid_new is unique key; q12 and pppartyid3 are political party related questions; q23 is income gap

columns_to_keep = ['caseid_new','q12','pppartyid3','q23']

df = raw[columns_to_keep]

df.head()

,caseid_new,q12,pppartyid3,q23
0,22526,democrat,democrat,partner earned more
1,23286,democrat,democrat,i earned more
2,26315,democrat,democrat,i earned more
3,28536,democrat,democrat,i earned more
4,29584,democrat,democrat,partner earned more


## Same Political

In [7]:
%%time
df['same_political'] = np.NaN

Wall time: 25.9 ms


C:\Users\xiex8\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
def same_political (row):
    respondent = row['pppartyid3']
    partner=row['q12']
    
    if respondent == partner:
        row['same_political'] = 1
    else:
        row['same_political'] = 0
    
    return row

In [9]:
df = df.apply(same_political, axis=1)

In [10]:
df.head()

,caseid_new,q12,pppartyid3,q23,same_political
0,22526,democrat,democrat,partner earned more,1
1,23286,democrat,democrat,i earned more,1
2,26315,democrat,democrat,i earned more,1
3,28536,democrat,democrat,i earned more,1
4,29584,democrat,democrat,partner earned more,1


In [11]:
df['same_political'].value_counts()

1    1738
0    1271
Name: same_political, dtype: int64

## Income Gap

In [12]:
df['q23'].value_counts()

i earned more                      1332
partner earned more                1272
we earned about the same amount     374
refused                              31
Name: q23, dtype: int64

In [13]:
%%time
df['income_gap'] = np.NaN
df['respondent_earn_more'] = np.NaN

Wall time: 1 ms


In [14]:
def income_gap (row):
    
    if row['q23'] == 'we earned about the same amount':
        row['income_gap'] = 0
        
    elif row['q23'] == 'refused':
        row['income_gap'] = np.NaN
    
    else:
        row['income_gap'] = 1
        
        if row['q23'] == 'i earned more':
            row['respondent_earn_more'] = 1
        else:
            row['respondent_earn_more'] = 0
    
    return row

In [15]:
df = df.apply(income_gap, axis=1)

In [16]:
df.head()

,caseid_new,q12,pppartyid3,q23,same_political,income_gap,respondent_earn_more
0,22526,democrat,democrat,partner earned more,1,1.0,0.0
1,23286,democrat,democrat,i earned more,1,1.0,1.0
2,26315,democrat,democrat,i earned more,1,1.0,1.0
3,28536,democrat,democrat,i earned more,1,1.0,1.0
4,29584,democrat,democrat,partner earned more,1,1.0,0.0


In [17]:
df['income_gap'].value_counts()

1.0    2604
0.0     374
Name: income_gap, dtype: int64

In [18]:
df['respondent_earn_more'].value_counts()

1.0    1332
0.0    1272
Name: respondent_earn_more, dtype: int64

## Final clean up for "income gap" and "same political" and save to CSV

In [19]:
df = df.rename(columns={
    'q12':'partner_political',
    'pppartyid3':'resondent_political',
    'q23':'income_gap_raw'
})

In [20]:
df.shape

(3009, 7)

In [21]:
df.columns

Index(['caseid_new', 'partner_political', 'resondent_political',
       'income_gap_raw', 'same_political', 'income_gap',
       'respondent_earn_more'],
      dtype='object')

In [22]:
pd.pivot_table(df,values="caseid_new",index="income_gap_raw",aggfunc='count',dropna=True, fill_value=0)

,caseid_new
income_gap_raw,
i earned more,1332
partner earned more,1272
refused,31
we earned about the same amount,374


In [23]:
df.to_csv('project_3_political_income_gap.csv')

## Create dummy for all the categoricals in the project 2 final table

In [24]:
prjt2_df = pd.read_csv("project_2_final_table.csv")

In [25]:
prjt2_df.columns

Index(['Unnamed: 0', 'caseid_new', 'qflag_w1', 'married_w1', 'children_in_hh',
       'age_difference', 'age_gap_bin', 'same_sex_couple', 'race_gap',
       'religious_gap', 'edu_gap', 'edu_gap_bin', 'parental_approval',
       'met_online', 'met_at_work', 'met_at_school', 'met_at_church',
       'met_travel', 'met_social', 'met_party', 'met_f_and_f',
       'met_as_neighbors', 'met_public_space', 'met_offline_dating',
       'met_other', 'how_met_count', 'how_met_unique',
       'how_long_relationship_w1', 'relationship_len', 'partner_deceased',
       'relationship_quality_w1', 'qflag_w2', 'cflag_w2',
       'relationship_status_w2', 'qflag_w3', 'cflag_w3',
       'relationship_status_w3', 'qflag_w4', 'cflag_w4',
       'relationship_status_w4', 'qflag_w5', 'cflag_w5',
       'relationship_status_w5', 'qflag_w6', 'cflag_w6',
       'relationship_status_w6'],
      dtype='object')

In [26]:
# create a list for all the columns to keep for project 3 feature engineering

cls_to_keep = ['caseid_new','married_w1', 'children_in_hh',
       'age_difference', 'age_gap_bin', 'same_sex_couple', 'race_gap',
       'religious_gap', 'edu_gap', 'edu_gap_bin', 'parental_approval',
       'met_online', 'met_at_work', 'met_at_school', 'met_at_church',
       'met_travel', 'met_social', 'met_party', 'met_f_and_f',
       'met_as_neighbors', 'met_public_space', 'met_offline_dating',
       'met_other', 'how_met_count', 'relationship_len', 'partner_deceased',
       'relationship_quality_w1']

In [27]:
# drop uncenessary columns

prjt2_df = prjt2_df[cls_to_keep]
prjt2_df.head()

,caseid_new,married_w1,children_in_hh,age_difference,age_gap_bin,same_sex_couple,race_gap,religious_gap,edu_gap,edu_gap_bin,...,met_party,met_f_and_f,met_as_neighbors,met_public_space,met_offline_dating,met_other,how_met_count,relationship_len,partner_deceased,relationship_quality_w1
0,22526,not married,0,4.0,4 to 5,same-sex couple,1,1,2.0,0 to 2,...,0,0,0,1,0,0,2,12.0,0,good
1,23286,married,0,2.0,0 to 3,same-sex couple,0,1,1.0,0 to 2,...,0,1,1,0,0,0,3,13.0,0,good
2,26315,not married,0,9.0,6 to 10,same-sex couple,0,1,2.0,0 to 2,...,0,1,0,0,0,0,2,8.0,0,good
3,28536,not married,0,2.0,0 to 3,same-sex couple,0,1,1.0,0 to 2,...,0,0,0,1,0,0,1,17.0,0,good
4,29584,married,0,7.0,6 to 10,different sex couple,0,0,0.0,0 to 2,...,0,0,0,0,0,0,1,35.0,0,good


In [28]:
##########################################################################################
###### check a couple catergorical columns and see if they are string or numerical  ######
##########################################################################################


prjt2_df['parental_approval'].value_counts() # string
# approve                        1643
# don't approve or don't know     460

prjt2_df['married_w1'].value_counts() # string
# married        1928
# not married    1081

prjt2_df['same_sex_couple'].value_counts() # string
# different sex couple    2535
# same-sex couple          474

prjt2_df['edu_gap_bin'].value_counts() # string
# 0 to 2    2116
# 3 to 4     653
# 5 to 6     130
# >6         102

prjt2_df['relationship_quality_w1'].value_counts() # string
# excellent    1771
# good          911
# fair          252
# poor           42
# very poor      20

excellent    1771
good          911
fair          252
poor           42
very poor      20
Name: relationship_quality_w1, dtype: int64

In [29]:
##############################################################################################################
## create dummy for all categorical variables. For 2-value-only variables will remove excessive columns 
##############################################################################################################

## https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

prjt2_df = pd.get_dummies(prjt2_df,columns=['married_w1','age_gap_bin','same_sex_couple','edu_gap_bin','parental_approval','relationship_quality_w1'])

In [30]:
prjt2_df = prjt2_df.rename(columns={
    'parental_approval_don\'t approve or don\'t know':"parental_approval_dont_approve"
})

In [31]:
prjt2_df.columns

Index(['caseid_new', 'children_in_hh', 'age_difference', 'race_gap',
       'religious_gap', 'edu_gap', 'met_online', 'met_at_work',
       'met_at_school', 'met_at_church', 'met_travel', 'met_social',
       'met_party', 'met_f_and_f', 'met_as_neighbors', 'met_public_space',
       'met_offline_dating', 'met_other', 'how_met_count', 'relationship_len',
       'partner_deceased', 'married_w1_married', 'married_w1_not married',
       'age_gap_bin_0 to 3', 'age_gap_bin_11 to 20', 'age_gap_bin_4 to 5',
       'age_gap_bin_6 to 10', 'age_gap_bin_>=20',
       'same_sex_couple_different sex couple',
       'same_sex_couple_same-sex couple', 'edu_gap_bin_0 to 2',
       'edu_gap_bin_3 to 4', 'edu_gap_bin_5 to 6', 'edu_gap_bin_>6',
       'parental_approval_approve', 'parental_approval_dont_approve',
       'relationship_quality_w1_excellent', 'relationship_quality_w1_fair',
       'relationship_quality_w1_good', 'relationship_quality_w1_poor',
       'relationship_quality_w1_very poor'],
  

In [32]:
prjt2_df.head()

,caseid_new,children_in_hh,age_difference,race_gap,religious_gap,edu_gap,met_online,met_at_work,met_at_school,met_at_church,...,edu_gap_bin_3 to 4,edu_gap_bin_5 to 6,edu_gap_bin_>6,parental_approval_approve,parental_approval_dont_approve,relationship_quality_w1_excellent,relationship_quality_w1_fair,relationship_quality_w1_good,relationship_quality_w1_poor,relationship_quality_w1_very poor
0,22526,0,4.0,1,1,2.0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,23286,0,2.0,0,1,1.0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
2,26315,0,9.0,0,1,2.0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,28536,0,2.0,0,1,1.0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,29584,0,7.0,0,0,0.0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0


In [33]:
prjt2_df.to_csv('project_3_dummied_prjt2_clmns_v2.csv')

## Merge Megan, Amanda and my new columns

In [34]:
megan = pd.read_csv("project_3_age_family_region.csv")
elle = pd.read_csv('project_3_political_income_gap.csv')
amanda = pd.read_csv('project_3_relationship_cat.csv')

In [35]:
megan.head()

,Unnamed: 0,caseid_new,age_when_met,family_size,region,metro,region_midwest,region_northeast,region_south,region_west
0,0,22526,45.0,0.0,2,1,1,0,0,0
1,1,23286,19.0,0.0,1,1,0,0,0,1
2,2,26315,23.0,1.0,0,1,0,0,1,0
3,3,28536,41.0,1.0,2,1,1,0,0,0
4,4,29584,28.0,0.0,0,1,0,0,1,0


In [36]:
elle.head()

,Unnamed: 0,caseid_new,partner_political,resondent_political,income_gap_raw,same_political,income_gap,respondent_earn_more
0,0,22526,democrat,democrat,partner earned more,1,1.0,0.0
1,1,23286,democrat,democrat,i earned more,1,1.0,1.0
2,2,26315,democrat,democrat,i earned more,1,1.0,1.0
3,3,28536,democrat,democrat,i earned more,1,1.0,1.0
4,4,29584,democrat,democrat,partner earned more,1,1.0,0.0


In [37]:
amanda.head()

,Unnamed: 0,caseid_new,relationship_len,partner_deceased,relationship_quality_w1,category
0,0,22526,12.0,0,good,second_quartile
1,1,23286,13.0,0,good,second_quartile
2,2,26315,8.0,0,good,bottom_quartile
3,3,28536,17.0,0,good,second_quartile
4,4,29584,35.0,0,good,top_quartile


In [38]:
df1 = pd.merge(prjt2_df,megan,how='left',on='caseid_new')
df2 = pd.merge(df1,elle,how='left',on='caseid_new')
final_df = pd.merge(df2,amanda,how='left',on='caseid_new')

del final_df['Unnamed: 0_x']
del final_df['Unnamed: 0_y']
del final_df['Unnamed: 0']
del final_df['relationship_len_y']
del final_df['partner_deceased_y']


final_df.columns

Index(['caseid_new', 'children_in_hh', 'age_difference', 'race_gap',
       'religious_gap', 'edu_gap', 'met_online', 'met_at_work',
       'met_at_school', 'met_at_church', 'met_travel', 'met_social',
       'met_party', 'met_f_and_f', 'met_as_neighbors', 'met_public_space',
       'met_offline_dating', 'met_other', 'how_met_count',
       'relationship_len_x', 'partner_deceased_x', 'married_w1_married',
       'married_w1_not married', 'age_gap_bin_0 to 3', 'age_gap_bin_11 to 20',
       'age_gap_bin_4 to 5', 'age_gap_bin_6 to 10', 'age_gap_bin_>=20',
       'same_sex_couple_different sex couple',
       'same_sex_couple_same-sex couple', 'edu_gap_bin_0 to 2',
       'edu_gap_bin_3 to 4', 'edu_gap_bin_5 to 6', 'edu_gap_bin_>6',
       'parental_approval_approve', 'parental_approval_dont_approve',
       'relationship_quality_w1_excellent', 'relationship_quality_w1_fair',
       'relationship_quality_w1_good', 'relationship_quality_w1_poor',
       'relationship_quality_w1_very poor',

In [39]:
final_df = final_df.rename(columns={
    'category':'relationship_len_cat',
    'relationship_len_x':'relationship_len',
    'partner_deceased_x':'partner_deceased'
})

final_df.columns

Index(['caseid_new', 'children_in_hh', 'age_difference', 'race_gap',
       'religious_gap', 'edu_gap', 'met_online', 'met_at_work',
       'met_at_school', 'met_at_church', 'met_travel', 'met_social',
       'met_party', 'met_f_and_f', 'met_as_neighbors', 'met_public_space',
       'met_offline_dating', 'met_other', 'how_met_count', 'relationship_len',
       'partner_deceased', 'married_w1_married', 'married_w1_not married',
       'age_gap_bin_0 to 3', 'age_gap_bin_11 to 20', 'age_gap_bin_4 to 5',
       'age_gap_bin_6 to 10', 'age_gap_bin_>=20',
       'same_sex_couple_different sex couple',
       'same_sex_couple_same-sex couple', 'edu_gap_bin_0 to 2',
       'edu_gap_bin_3 to 4', 'edu_gap_bin_5 to 6', 'edu_gap_bin_>6',
       'parental_approval_approve', 'parental_approval_dont_approve',
       'relationship_quality_w1_excellent', 'relationship_quality_w1_fair',
       'relationship_quality_w1_good', 'relationship_quality_w1_poor',
       'relationship_quality_w1_very poor', 'ag

In [40]:
final_df.to_csv('project_3_final_table.csv')